In [ ]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns

from amftrack.util.dbx import upload_folders, load_dbx, download
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import (
    read_saved_dropbox_state,
    get_dropbox_folders_prince,
    get_dropbox_folders_prince,
    save_dropbox_state,
    get_dropbox_folders_general_recursive,
)

In [ ]:
all_folders_drop = get_dropbox_folders_prince("/DATA/MYRISTATE/PRINCE")
# all_folders_drop = get_dropbox_folders_prince("/DATA/PRINCE")


In [ ]:
plates = [
1,
49,
51,
14,
58,
59,
60,
31,
33,
34,
62,
63,
64,
67,
68,
69,
72,
37,
46,
48,
]

In [ ]:
plates = [str(plate) for plate in plates]

In [ ]:
folders = all_folders_drop[all_folders_drop['Plate'].isin(plates)]
folders_drop = folders[folders['CrossDate']>'20230101']

In [ ]:
folders_drop.groupby('unique_id')['folder'].count()

In [ ]:
plates = ['58_20230511', '62_20230511', '1_20230221', '63_20230510',
       '51_20230510', '49_20230510', '64_20230508', '60_20230508',
       '59_20230508', '72_20230501', '69_20230501', '67_20230501',
       '68_20230501', '37_20230221', '14_20230301', '34_20230227',
       '31_20230227', '48_20230227', '33_20230221', '46_20230221']

In [ ]:
folders_drop = all_folders_drop[all_folders_drop['unique_id'].isin(plates)]

In [ ]:
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(all_folders["folder"])]
folders_drop2

In [ ]:
directory_targ = directory_project+'myr/'
# directory_targ = directory_targ+'myr/'
# directory_targ

In [ ]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop2,
    40,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="fig1",
)

In [ ]:
update_plate_info(directory_targ, local=True,strong_constraint=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
eval("False")

In [ ]:
dir_drop = "DATA/MYRISTATE/PRINCE"
delete = False
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    all_folders,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="correct_myr_files",
)

To correct maforme Img paths

In [ ]:
update_plate_info(directory_targ, local=True,strong_constraint=False)
wrong_folder = get_current_folders(directory_targ, local=True)

In [ ]:
wrong_folder

In [ ]:
for folder_path in wrong_folder['total_path']:

    # Check if it's a directory and contains an 'Img' subfolder
    imgimg_subfolder_path = os.path.join(folder_path, "Img","Img")
    if os.path.isdir(folder_path) and os.path.isdir(imgimg_subfolder_path):
        
        # Move each file from the 'Img' subfolder to its parent folder
        for file in os.listdir(imgimg_subfolder_path):
            final_file_path = os.path.join(folder_path, "Img",file)
            init_file_path = os.path.join(imgimg_subfolder_path,file)
            print(init_file_path,final_file_path)
            shutil.move(init_file_path, final_file_path)

        # Remove the now empty 'Img' subfolder
        os.rmdir(imgimg_subfolder_path)